In [1]:
import os

os.environ["CUDA_VISIBLE_DEVICES"]= "1" 

In [3]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, DynamicCache
from baukit import TraceDict
from tqdm import tqdm

model_name = "meta-llama/Meta-Llama-3.1-8B-Instruct"
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained(model_name)

device = torch.device("cuda")
model.to(device)

# 원래 프롬프트 준비 및 토큰화
input_text = "You are an intelligent assistant trained to classify customer queries into predefined categories. Your task is to read a customer's query and assign it to the most appropriate category from the following list of categories:\n1.activate_my_card\n2.age_limit\n3.apple_pay_or_google_pay\n4.atm_support\n5.automatic_top_up\n6.balance_not_updated_after_bank_transfer\n7.balance_not_updated_after_cheque_or_cash_deposit\n8.beneficiary_not_allowed\n9.cancel_transfer\n10.card_about_to_expire\n11.card_acceptance\n12.card_arrival\n13.card_delivery_estimate\n14.card_linking\n15.card_not_working\n16.card_payment_fee_charged\n17.card_payment_not_recognised\n18.card_payment_wrong_exchange_rate\n19.card_swallowed\n20.cash_withdrawal_charge\n21.cash_withdrawal_not_recognised\n22.change_pin\n23.compromised_card\n24.contactless_not_working\n25.country_support\n26.declined_card_payment\n27.declined_cash_withdrawal\n28.declined_transfer\n29.direct_debit_payment_not_recognised\n30.disposable_card_limits\n31.edit_personal_details\n32.exchange_charge\n33.exchange_rate\n34.exchange_via_app\n35.extra_charge_on_statement\n36.failed_transfer\n37.fiat_currency_support\n38.get_disposable_virtual_card\n39.get_physical_card\n40.getting_spare_card\n41.getting_virtual_card\n42.lost_or_stolen_card\n43.lost_or_stolen_phone\n44.order_physical_card\n45.passcode_forgotten\n46.pending_card_payment\n47.pending_cash_withdrawal\n48.pending_top_up\n49.pending_transfer\n50.pin_blocked\n51.receiving_money\n52.refund_not_showing_up\n53.request_refund\n54.reverted_card_payment\n55.supported_cards_and_currencies\n56.terminate_account\n57.top_up_by_bank_transfer_charge\n58.top_up_by_card_charge\n59.top_up_by_cash_or_cheque\n60.top_up_failed\n61.top_up_limits\n62.top_up_reverted\n63.topping_up_by_card\n64.transaction_charged_twice\n65.transfer_fee_charged\n66.transfer_into_account\n67.transfer_not_received_by_recipient\n68.transfer_timing\n69.unable_to_verify_identity\n70.verify_my_identity\n71.verify_source_of_funds\n72.verify_top_up\n73.virtual_card_not_working\n74.visa_or_mastercard\n75.why_verify_identity\n76.wrong_amount_of_cash_received\n77.wrong_exchange_rate_for_cash_withdrawal\nPlease analyze the query and return the most suitable category from the list above. If the query does not fit perfectly into one category, select the category that best matches the main topic of the query. Only return intent category text and don't return any other words or numbers.\nQuery: I broke my card\nIntent:"
inputs = tokenizer(input_text, return_tensors="pt").to(device)
input_ids = inputs.input_ids
original_prompt_length = input_ids.shape[1]

intervention_vectors = torch.load("./results_bf16/banking77/42/stacked_diff_icv.pt").to(dtype=torch.bfloat16).to(device)

def add_icv(edit_layer, icv, device, idx=-1):
    def add_act(output, layer_name):
        current_layer = int(layer_name.split(".")[2])
        if current_layer in edit_layer:
            if isinstance(output, tuple):
                output[0][:, idx] += icv[current_layer].to(device)
                return output
            else:
                return output
        else:
            return output
    return add_act

last_idx = len(inputs.input_ids[0])-1
intervention_fn= add_icv(list(range(32)), intervention_vectors, device, idx=last_idx)

for _ in tqdm(range(500)):
    with TraceDict(model, layers = [f'model.layers.{layer}' for layer in range(model.config.num_hidden_layers)], edit_output=intervention_fn):
        p_output = model.forward(**inputs)
        new_token_ids = torch.argmax(p_output.logits[:,-1,:])
        if new_token_ids == tokenizer.eos_token_id or new_token_ids==tokenizer.encode('\n\n', add_special_tokens=False)[0] or new_token_ids==tokenizer.encode('\n', add_special_tokens=False)[0]:
            break
        new_token_ids = new_token_ids.view(1,-1)
        print(new_token_ids)
        inputs.input_ids = torch.cat((inputs.input_ids, new_token_ids), dim=-1)


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

/tmp/ipykernel_850595/693442787.py:19: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  intervention_vectors = torch.load("./results_bf16/banking77/42/stacked_diff_icv.pt").to(

FileNotFoundError: [Errno 2] No such file or directory: './results_bf16/banking77/42/stacked_diff_icv.pt'